In [1]:
import jn_setup
from tools.paths import *
from simulation_procedure import make_model
import pandas as pd, pyabc, hydroeval as he, numpy as np, uuid

In [2]:
# mode = "test"
mode = "debug"
# debug_params = [1,2,3,4,5,16,17,18,19,20] # if given, use subset of parameters instead of all parameters
debug_params = [] # if given empty list, don't subset parameters. Use all parameters.
model = make_model(mode = mode, swmm_cleanup = 'full', vvwm_cleanup = 'full')

C:\Users\jstelman\Git\stelman_urban_pesticides\master_debug\NPlesantCreek.inp


In [3]:
# Priors. Get the values from that csv. Just for SWMM at first.
swmm_ranges = pd.read_csv(os.path.join(master_path, "swmm_param_priors.csv"), index_col=0,
                           usecols = ["Parameter", "Min", "Range"])
#Link up with the vvwm priors and make one big list with 34 params
vvwm_ranges = pd.read_csv(os.path.join(master_path, "vvwm_param_priors.csv"), index_col=0,
                           usecols = ["Parameter", "Min", "Range"])
param_ranges = pd.concat([swmm_ranges, vvwm_ranges], axis = 0)

# if subset specified, subset parameters forming joint prior as specified
if mode == 'debug' and debug_params:
    param_ranges = param_ranges.iloc[debug_params]
# make a dictionary
priors = param_ranges.to_dict("index")
# make prior object
# borrowed from Jeff: <https://github.com/JeffreyMinucci/bee_neonic_abc/blob/master/pyabc_run.ipynb>
prior = pyabc.Distribution(**{key: pyabc.RV("uniform", loc = v['Min'], scale = v['Range'])
                        for key, v in priors.items()})

In [4]:
priors

{'NImperv': {'Min': 0.01, 'Range': 0.015},
 'NPerv': {'Min': 0.05, 'Range': 0.45},
 'SImperv': {'Min': 1.27, 'Range': 1.27},
 'SPerv': {'Min': 2.54, 'Range': 2.54},
 'PctZero': {'Min': 0.01, 'Range': 99.99},
 'MaxRate': {'Min': 8.46, 'Range': 118.54},
 'MinRate': {'Min': 0.254, 'Range': 10.666},
 'Decay': {'Min': 2.0, 'Range': 5.0},
 'DryTime': {'Min': 2.0, 'Range': 12.0},
 'Por': {'Min': 0.4, 'Range': 0.1},
 'WP': {'Min': 0.024, 'Range': 0.241},
 'FC': {'Min': 0.06, 'Range': 0.32},
 'Ksat': {'Min': 0.25, 'Range': 12.45},
 'Kdecay': {'Min': 0.002739726, 'Range': 0.197260274},
 'BCoeff2': {'Min': 0.5, 'Range': 1.5},
 'WCoeff2': {'Min': 0.066, 'Range': 0.148},
 'kd': {'Min': 882.0, 'Range': 5028.0},
 'aer_aq': {'Min': 5.0, 'Range': 360.0},
 'aer_aq_temp': {'Min': 20.0, 'Range': 5.0},
 'anae_aq': {'Min': 5.0, 'Range': 725.0},
 'anae_aq_temp': {'Min': 20.0, 'Range': 5.0},
 'photo': {'Min': 96.9, 'Range': 319.1},
 'hydro': {'Min': 0.1, 'Range': 364.9},
 'sol': {'Min': 0.00024063, 'Range': 0

In [5]:
prior

<Distribution 'BCoeff2', 'Decay', 'DryTime', 'FC', 'Kdecay', 'Ksat', 'MaxRate', 'MinRate', 'NImperv', 'NPerv', 'PctZero', 'Por', 'SImperv', 'SPerv', 'WCoeff2', 'WP', 'aer_aq', 'aer_aq_temp', 'anae_aq', 'anae_aq_temp', 'benthic_depth', 'bnmas', 'bulk_density', 'chl', 'doc1', 'doc2', 'froc1', 'froc2', 'hydro', 'kd', 'photo', 'porosity', 'sol', 'sused'>

# Make the .new object
### 1. Import observed data

In [6]:
if mode == 'debug':
    with open(os.path.join(main_path, 'master_debug','debug_obs_data.txt'),'r') as read_file:
        obs_dict = eval(read_file.read())
elif mode == 'test':
    with open(os.path.join(main_path, 'master_test','test_obs_data.txt'),'r') as read_file:
        obs_dict = eval(read_file.read())
elif mode == 'run':
    with open(os.path.join(main_path, 'master','obs_data.txt'),'r') as read_file:
        obs_dict = eval(read_file.read())
obs_dict

{'2009-02-13_28': 0.0485, '2009-04-07_28': 0.0192, '2009-04-13_28': 0.00858}

### 2. Initialize dask client for dask distributed sampler

In [7]:
from dask.distributed import Client#, LocalCluster
# cluster = LocalCluster()#n_workers=(90/2), threads_per_worker = 2)  # Set for 96 vCPU compute instance
# client = Client(cluster)#,timeout=400)

# make it simpler
# if __name__ == "__main__":
client = Client()
# make sampler object
sampler = pyabc.sampler.DaskDistributedSampler(dask_client = client)

# # See if this takes all those errors out
# sampler = pyabc.sampler.SingleCoreSampler()

# # make the process more transparent
sampler.sample_factory.record_rejected = True
sampler.show_progress = True

### 3. Set up a sqlite db directory

In [8]:
# Initialize a new ABC inference run
# create a random, unique database ID
dbid = uuid.uuid4().hex[0:8]
print(dbid)
# compose path to run-specific database
database_dir = os.path.join(temp_path, 'results_db')  
if not os.path.exists(database_dir):
    os.mkdir(database_dir)
db_path = ("sqlite:///" +
           os.path.join(database_dir, "test_pyabc_" + dbid + ".db"))

04c40643


### 4. Defining a Distance function

We need to refactor the NSE distance function using the pyabc.Distance class.
We will need the hydroeval library and the pyabc.SimpleFunctionDistance to do this

In [9]:
# make a file to hold onto these NSEs for our own record
with open(os.path.join(temp_path, "NSEs_" + dbid + ".txt"), "w") as nse_file:
    nse_file.write("NSEs\n")

In [10]:
# define NSE Distance function: Calculate NSE with hydroeval library and the subtract 1 from it to get NSED
def nse(x, x_0):
    nse = he.evaluator(he.nse, 
                       simulation_s = np.array(list(x.values())),
                       evaluation = np.array(list(x_0.values())))[0]
    print("nse ", nse)
    # make record
    with open(os.path.join(temp_path, "NSEs_" + dbid + ".txt"),"a") as nse_file:
        nse_file.write(str(nse)+"\n")
    return nse
    
# NSE = pyabc.SimpleFunctionDistance(fun=nse)

# the best answer is 1
# make one that measures distance from 1
NSED = pyabc.SimpleFunctionDistance(fun = lambda x, x_0: (1 - nse(x, x_0)))

### 5. Define ABCSMC object

In [11]:
abc = pyabc.ABCSMC(model, prior, 
                   population_size = pyabc.ConstantPopulationSize(5), # just to shorten the run
                   sampler = sampler,
                   distance_function = NSED)

In [12]:
abc

### 6. Initialize a new abc run

In [13]:
abc.new(db_path, obs_dict)

In [14]:
# Back to 1 gen
history = abc.run(max_nr_populations=2, minimum_epsilon=0.2)

nse  -5.74001848760223e+22
nse  -7.733160114201307e+22
nse  -1.5345095643064505e+23
nse  -2.37602204687532e+22
nse  -6.560020838135587e+23


distributed.nanny - WARNING - Worker process still alive after 4 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 4 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 4 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 4 seconds, killing
